In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization,
    UpSampling1D, Reshape, Cropping1D, ZeroPadding1D, Add, LeakyReLU
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, Conv1DTranspose, BatchNormalization, Dropout, Activation,
    Add, Flatten, Dense, Reshape
)
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
import numpy as np

#***Load the filtered signals and split to train, test and validation:***

---



In [ ]:
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# Function to load signals from a folder
def load_filtered_signals_with_metadata_from_folder(folder_path, expected_length=5000):
    signals_with_metadata = []

    # List all .npy files in the folder
    file_list = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

    for file_name in file_list:
        try:
            file_path = os.path.join(folder_path, file_name)
            signal = np.load(file_path)
            first_dim = signal[:, 0] if signal.ndim == 2 else signal

            # Filter by length
            if len(first_dim) != expected_length:
                continue

            # Parse ID and lead
            file_name_wo_ext = os.path.splitext(file_name)[0]
            # file_id, lead = file_name_wo_ext.split('_', 1)
            parts = file_name.split('_')
            file_id = parts[0]
            lead = parts[5]

            signals_with_metadata.append((first_dim, file_id, lead))
        except Exception as e:
            print(f"Failed to load {file_name}: {e}")

    return signals_with_metadata

# Load the signals
data_directory = '/sise/nadav-group/nadavrap-group/ECGs/Final Project/Preprocessing/Filtered_Files_updated/'
signals_with_metadata = load_filtered_signals_with_metadata_from_folder(data_directory)

print(f"Total loaded valid signals: {len(signals_with_metadata)}")



# Split into train (70%), validation (10%), and test (20%)
train_data, test_data = train_test_split(signals_with_metadata, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.125, random_state=42)  # 10% of total for validation

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print("Test set size:", len(test_data))

# Optional: convert to arrays ready for Conv1D (if needed later)
train_signals = np.array([np.expand_dims(s[0], axis=-1) for s in train_data])
val_signals = np.array([np.expand_dims(s[0], axis=-1) for s in val_data])

print(f"train_signals shape: {train_signals.shape}")
print(f"val_signals shape: {val_signals.shape}")

#***Prepare training data + Normalization according to training signal statistics:***

---



In [ ]:
def prepare_training_data(signals_with_metadata):
    training_signals = np.array([s[0] for s in signals_with_metadata])
    metadata = [(s[1], s[2]) for s in signals_with_metadata]
    return training_signals, metadata


training_signals, metadata = prepare_training_data(train_data)
training_signals = np.expand_dims(training_signals, axis=-1)

validation_signals, val_metadata = prepare_training_data(val_data)
validation_signals = np.expand_dims(validation_signals, axis=-1)

global_mean = np.mean(training_signals)
global_std = np.std(training_signals)
# Prevent division by zero
if global_std == 0:
    global_std = 1
training_signals = (training_signals - global_mean) / global_std

validation_signals = (validation_signals - global_mean) / global_std

#***1D Residual CNN Autoencoder Model:***

---



In [ ]:
from tensorflow.keras.layers import (
    Input, Conv1D, UpSampling1D, BatchNormalization, Dropout,
    LeakyReLU, Flatten, Dense, Reshape, Add
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import Huber
import tensorflow as tf
import numpy as np

def residual_conv_block(x, filters, kernel_size=7, strides=1, reg=1e-4, dropout_rate=0.2):
    shortcut = x
    x = Conv1D(filters, kernel_size, strides=strides, padding='same', kernel_regularizer=l2(reg))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv1D(filters, kernel_size, strides=1, padding='same', kernel_regularizer=l2(reg))(x)
    x = BatchNormalization()(x)

    # Adjust shortcut shape if needed
    if shortcut.shape[-1] != filters or strides != 1:
        shortcut = Conv1D(filters, 1, strides=strides, padding='same', kernel_regularizer=l2(reg))(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = LeakyReLU()(x)

    if dropout_rate > 0:
        x = Dropout(dropout_rate)(x)
    return x

def upsample_block(x, filters, kernel_size=7, up_size=2, reg=1e-4):
    x = UpSampling1D(size=up_size)(x)
    x = Conv1D(filters, kernel_size, padding='same', kernel_regularizer=l2(reg))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    return x

def build_autoencoder_with_encoder_residuals(input_length=5000, latent_dim=32, reg=1e-4):
    input_signal = Input(shape=(input_length, 1))

    # Encoder with residual blocks
    x = residual_conv_block(input_signal, 32, strides=2, reg=reg)
    x = residual_conv_block(x, 64, strides=2, reg=reg)
    x = residual_conv_block(x, 128, strides=2, reg=reg)

    shape_before_flatten = tf.keras.backend.int_shape(x)[1:]
    flat = Flatten()(x)
    encoded = Dense(latent_dim, name="latent_vector")(flat)

    # Decoder (no residuals)
    x = Dense(np.prod(shape_before_flatten))(encoded)
    x = Reshape(target_shape=shape_before_flatten)(x)

    x = upsample_block(x, 128, up_size=2, reg=reg)
    x = upsample_block(x, 64, up_size=2, reg=reg)
    x = upsample_block(x, 32, up_size=2, reg=reg)

    decoded = Conv1D(1, kernel_size=3, padding='same', activation='linear')(x)

    autoencoder = Model(inputs=input_signal, outputs=decoded)
    # autoencoder.compile(optimizer=Adam(1e-3), loss=Huber())  # אפשר גם loss='mse'
    autoencoder.compile(optimizer=Adam(1e-3), loss='mse')
    return autoencoder

In [ ]:
autoencoder_v2 = build_autoencoder_with_encoder_residuals(latent_dim=32)

early_stop = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, verbose=1)

history = autoencoder_v2.fit(
    training_signals, training_signals,
    validation_data=(validation_signals, validation_signals),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr]
)

In [ ]:
val_reconstructed = autoencoder_v2.predict(validation_signals)
mse = np.mean((val_reconstructed - validation_signals) ** 2)
print(f"Validation MSE: {mse:.6f}")

In [ ]:
# Save the model
model_path = '/sise/nadav-group/nadavrap-group/ECGs/Final Project/Autoencoder/autoencode_selin_without_normalization_v2.keras'
autoencoder_v2.save(model_path)

In [ ]:
model_path = '/sise/nadav-group/nadavrap-group/ECGs/Final Project/Autoencoder/autoencode_selin_without_normalization_v2.keras'

loaded_model = tf.keras.models.load_model(model_path)
loaded_model.summary()

#***Plots:***

---



In [ ]:
plt.plot(history.history['loss'], label='Train MSE')
plt.plot(history.history['val_loss'], label='Val MSE')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.legend()
plt.title('Training vs Validation Loss')
plt.grid(True)
plt.show()

In [ ]:
n = 5
for i in range(n):
    plt.figure(figsize=(10, 2))
    plt.plot(validation_signals[i].squeeze(), label='Original')
    plt.plot(val_reconstructed[i].squeeze(), label='Reconstructed', alpha=0.7)
    plt.legend()
    plt.title(f"Signal {i}")
    plt.grid(True)
    plt.show()

#***Running the model through all the signal data for extracting the latent space:***

---



In [ ]:
from tensorflow.keras.models import load_model, Model
import numpy as np
import os

# Load the model from a SavedModel directory
model_path = '/sise/nadav-group/nadavrap-group/ECGs/Final Project/Autoencoder/autoencode_selin_without_normalization_v2.keras'
best_model = tf.keras.models.load_model(model_path)

# Create a model that outputs the latent space
latent_model = Model(inputs=best_model.input, outputs=best_model.get_layer("latent_vector").output)

# Directory to save the latent space files
latent_space_dir = '/sise/nadav-group/nadavrap-group/ECGs/Final Project/Autoencoder/Latent_Space_without_norm'
os.makedirs(latent_space_dir, exist_ok=True)

def split_signals_metadata(signals_with_metadata):
    """
    Split a list of tuples into separate arrays for signals and metadata.

    Parameters:
        signals_with_metadata (list): A list where each item is a tuple:
            (signal: np.array, signal_id: str/int, lead: str/int)

    Returns:
        signals (np.ndarray): Array of signal arrays.
        metadata (list of tuples): List of (signal_id, lead) pairs.
    """
    signals = np.array([s[0] for s in signals_with_metadata])
    metadata = [(s[1], s[2]) for s in signals_with_metadata]
    return signals, metadata

def save_latent_space_with_examples(signals, metadata):
    """
    Extract and save latent space vectors from the given signals using the autoencoder.

    If a file with the same name already exists, saves the new file with '_extra' appended to the filename
    to avoid overwriting existing files.

    Parameters:
        signals (np.ndarray): Array of input signals to encode.
        metadata (list of tuples): Metadata for each signal (signal_id, lead).
    """
    example_count = 0
    for i, signal in enumerate(signals):
        # Extract latent vector
        latent_vector = latent_model.predict(np.expand_dims(signal, axis=0))[0]

        # Construct filename from metadata
        signal_id, lead = metadata[i]
        filename = f"{signal_id}_{lead}.npy"
        filepath = os.path.join(latent_space_dir, filename)

        # If file exists, add '_extra' suffix to filename
        if os.path.exists(filepath):
            filename = f"{signal_id}_{lead}_extra.npy"
            filepath = os.path.join(latent_space_dir, filename)

        # Save latent vector as .npy
        np.save(filepath, latent_vector)

        # Print first 3 examples
        if example_count < 3:
            print(f"Example {example_count + 1}:")
            print(f"Metadata: ID={signal_id}, Lead={lead}")
            print(f"Latent space length: {len(latent_vector)}")
            print(f"Latent vector: {latent_vector}\n")
            example_count += 1

        print(f"Saved latent space for signal {i+1}/{len(signals)} to {filepath}")

# ---- Usage Example ----

all_signals, metadata = prepare_training_data(signals_with_metadata)
all_signals = np.expand_dims(all_signals, axis=-1)
all_signals = (all_signals - global_mean) / global_std
# Assuming `signals_with_metadata` is a list of (signal, id, lead) tuples
# signals, metadata = split_signals_metadata(signals_with_metadata)
# norm_signals = normalize_signals(signals)  # Assuming this function is defined elsewhere
save_latent_space_with_examples(all_signals, metadata)

#***Filtering the latent space files acoording MSE < 0.4:***

---



In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
import pandas as pd

# חיזוי על כל הדגימות
reconstructed = best_model.predict(all_signals)

# חישוב MSE לכל דגימה
mse_per_sample = np.mean((reconstructed - all_signals) ** 2, axis=(1, 2))  # axis 1,2 = לאורך הזמן ולערוצים

# יצירת טבלה עם mse ונתוני metadata תואמים
df = pd.DataFrame({
    'mse': mse_per_sample,
    'file_id': [meta[0] for meta in metadata],
    'lead': [meta[1] for meta in metadata]
})

# מיון לפי MSE מהנמוך לגבוה
df_sorted = df.sort_values(by='mse').reset_index(drop=True)
df_sorted

In [ ]:
low_mse_df = df_sorted[df_sorted['mse'] < 0.4].reset_index(drop=True)
low_mse_df

In [ ]:
import os
import shutil
import pandas as pd

# Input: sorted dataframe with low MSE entries
# Assume low_mse_df is already defined
# low_mse_df = df_sorted[df_sorted['mse'] < 0.3].reset_index(drop=True)

latent_dir = '/sise/nadav-group/nadavrap-group/ECGs/Final Project/Autoencoder/Latent_Space_without_norm'
dest_dir = "/sise/nadav-group/nadavrap-group/ECGs/Final Project/Autoencoder/Filtered_Low_MSE_without_norm"

# Create destination directory if it doesn't exist
os.makedirs(dest_dir, exist_ok=True)

# Track how many files we actually copy
copied_count = 0
not_found = []

# Loop through each row in the filtered dataframe
for _, row in low_mse_df.iterrows():
    file_id = str(row['file_id'])
    lead = str(row['lead'])

    filename = f"{file_id}_{lead}.npy"
    src_path = os.path.join(latent_dir, filename)
    dst_path = os.path.join(dest_dir, filename)

    if os.path.exists(src_path):
        shutil.copy2(src_path, dst_path)
        copied_count += 1
    else:
        not_found.append(filename)

# Report
print("✅ Finished copying.")
print(f"Files copied: {copied_count}")
print(f"Missing files (not found): {len(not_found)}")

if not_found:
    print("Example missing files:")
    for f in not_found[:5]:
        print(f)

#***Creating ECG Phenotype txt files:***

---



In [ ]:

import os
import numpy as np
from collections import defaultdict

input_folder = "/sise/nadav-group/nadavrap-group/ECGs/Final Project/Autoencoder/Filtered_Low_MSE_without_norm"
output_folder = "/sise/nadav-group/nadavrap-group/ECGs/Final Project/GWAS/ECG_PHENOTYPE_without_norm"
os.makedirs(output_folder, exist_ok=True)

lead_to_data = defaultdict(list)

for filename in os.listdir(input_folder):
    if filename.endswith('.npy') and 'extra' not in filename:
        parts = filename.replace('.npy', '').split('_')
        if len(parts) != 2:
            continue
        file_id, lead = parts
        vec_path = os.path.join(input_folder, filename)
        try:
            vec = np.load(vec_path)
            lead_to_data[lead].append((file_id, vec))
        except Exception as e:
            print(f"Error reading {filename}: {e}")

for lead, samples in lead_to_data.items():
    output_file = os.path.join(output_folder, f"ecg_phenotype_{lead}.txt")
    with open(output_file, 'w') as f:
        # Writing Headings
        header = ['IID'] + [f'feature_{i+1}' for i in range(len(samples[0][1]))]
        f.write('\t'.join(header) + '\n')

        # Writing Data
        for file_id, vec in samples:
            line = [file_id] + [f'{float(v):.8f}' for v in vec]
            f.write('\t'.join(line) + '\n')

#***Plots:***

---



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(df_sorted.index, df_sorted['mse'], marker='o', linestyle='-')
plt.title("MSE per Sample (Sorted)")
plt.xlabel("Sample Index (Sorted by MSE)")
plt.ylabel("MSE")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(df_sorted['mse'], bins=50, color='skyblue', edgecolor='black')
plt.title("Histogram of MSE Values per Sample")
plt.xlabel("MSE")
plt.ylabel("Number of Samples")
plt.grid(True)
plt.tight_layout()
plt.show()
